# Get qSV-regressed clean data for SCZ cases

Initial purpose is to check whether AHBA axes help parse heterogeneity

In [3]:
source("../code/preprocess.R")

Loading required package: rafalib

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats

## Understanding data...

Data received from LIBD (has clinical data)

In [ ]:
load("../data/rse_gene_BrainSeq_phases_1_2.n1627.Rdata")

In [49]:
colData(rse_gene) %>% data.frame %>% group_by(Dataset,Dx) %>% 
summarise(n_samples=n()) %>% 
mutate(Dx = factor(Dx, levels=c('Control', 'SCZD', 'MDD', 'Bipolar'), ordered=T)) %>% arrange(Dataset, Dx)
# summarise(samples=n_distinct(RNum), donors = n_distinct(BrNum))

`summarise()` has grouped output by 'Dataset'. You can override using the
`.groups` argument.


Dataset,Dx,n_samples
<chr>,<ord>,<int>
BrainSeq_Phase1,Control,332
BrainSeq_Phase1,SCZD,178
BrainSeq_Phase1,MDD,150
BrainSeq_Phase1,Bipolar,67
BrainSeq_Phase2_DLPFC,Control,300
BrainSeq_Phase2_DLPFC,SCZD,152
BrainSeq_Phase2_DLPFC,MDD,1
BrainSeq_Phase2_HIPPO,Control,314
BrainSeq_Phase2_HIPPO,SCZD,132


In [25]:
library(readxl)
clinical <- read_excel("../data/Clin fields_ADS_subjects for UCAM collab_shared 2022.xlsx")

In [18]:
colData(rse_gene)$BrNum %>% unique %>% length

[1] 861

In [48]:
colData(rse_gene) %>% data.frame %>% select(BrNum, Dataset, Region, Dx) %>% 
group_by(Dx, BrNum) %>% 
# mutate(
#     n=n()
# ) %>%
summarise(
    Phase1_DLPFC=sum(Dataset=='BrainSeq_Phase1'),
    Phase2_DLPFC=sum(Dataset=='BrainSeq_Phase2_DLPFC'),
    Phase2_HIPPO=sum(Dataset=='BrainSeq_Phase2_HIPPO')
) %>% 
group_by(Dx,Phase1_DLPFC,Phase2_DLPFC, Phase2_HIPPO) %>% 
summarize(n_donors=n()) %>% 
mutate(Dx = factor(Dx, levels=c('Control', 'SCZD', 'MDD', 'Bipolar'), ordered=T)) %>% 
arrange(Dx, -row_number())
# filter(Dx=='MDD', n>1)
# group_by(Dataset, Region, n) %>% count()
# filter(n==3) %>% arrange(BrNum) %>% head

`summarise()` has grouped output by 'Dx'. You can override using the `.groups`
argument.
`summarise()` has grouped output by 'Dx', 'Phase1_DLPFC', 'Phase2_DLPFC'. You
can override using the `.groups` argument.


Dx,Phase1_DLPFC,Phase2_DLPFC,Phase2_HIPPO,n_donors
<ord>,<int>,<int>,<int>,<int>
Control,1,1,1,188
Control,1,1,0,43
Control,1,0,1,46
Control,1,0,0,55
Control,0,1,1,52
Control,0,1,0,17
Control,0,0,1,28
SCZD,1,1,1,87
SCZD,1,1,0,36


In [53]:
clinical_brains <- paste0('Br',clinical$`Br#`)
expression_brains <- colData(rse_gene)$BrNum

data.frame(brain=union(clinical_brains, expression_brains)) %>% 
mutate(clinical = brain %in% clinical_brains,
       expression = brain %in% expression_brains) %>% 
count(clinical, expression)

clinical,expression,n
<lgl>,<lgl>,<int>
FALSE,TRUE,1
TRUE,FALSE,98
TRUE,TRUE,860


In [123]:
rse_gene %>% colData %>% colnames

[1] "SAMPLE_ID"         "RNum"              "RIN"              
 [4] "Region"            "Dataset"           "BrNum"            
 [7] "Dx"                "Age"               "Sex"              
[10] "Race"              "Protocol"          "numReads"         
[13] "numMapped"         "numUnmapped"       "mitoMapped"       
[16] "totalMapped"       "overallMapRate"    "concordMapRate"   
[19] "mitoRate"          "rRNA_rate"         "totalAssignedGene"
[22] "bamFile"

Downloadable Phase1 data (has some clinical data)

In [111]:
load("../data/rse_gene_BrainSeq_Phase1_hg19_TopHat2_EnsemblV75.rda")
# tx <- load("../data/transcript_data_filtered_n495.rda")

In [112]:
colData(rse_gene) %>% data.frame %>% group_by(Dx) %>% count()

Dx,n
<chr>,<int>
Control,320
Schizo,175


In [90]:
tFpkm %>% dim

[1] 188578    495

Downloadable Phase2 data

In [50]:
load("../data/rse_gene_unfiltered.Rdata")
load("../data/rse_tx_unfiltered.Rdata")

In [51]:
colData(rse_gene) %>% data.frame %>% group_by(Region,Dx) %>% count

Region,Dx,n
<chr>,<chr>,<int>
DLPFC,Control,300
DLPFC,Schizo,153
HIPPO,Control,314
HIPPO,Schizo,133


In [5]:
load("../data/methprop_pd.Rdata")

In [21]:
pd %>% data.frame %>% colnames

[1] "SAMPLE_ID"              "FQCbasicStats"          "perBaseQual"           
 [4] "perTileQual"            "perSeqQual"             "perBaseContent"        
 [7] "GCcontent"              "Ncontent"               "SeqLengthDist"         
[10] "SeqDuplication"         "OverrepSeqs"            "AdapterContent"        
[13] "KmerContent"            "percentGC_R1"           "phred100_R1"           
[16] "phredGT30_R1"           "phredGT35_R1"           "Adapter88_R1"          
[19] "percentGC_R2"           "phred100_R2"            "phredGT30_R2"          
[22] "phredGT35_R2"           "Adapter88_R2"           "ERCCsumLogErr"         
[25] "bamFile"                "trimmed"                "numReads"              
[28] "numMapped"              "numUnmapped"            "overallMapRate"        
[31] "concordMapRate"         "totalMapped"            "mitoMapped"            
[34] "mitoRate"               "totalAssignedGene"      "rRNA_rate"             
[37] "RNum"                   "BrNum"                  "Region"                
[40] "RIN"                    "Age"                    "Sex"                   
[43] "Race"                   "Dx"                     "snpPC1"                
[46] "snpPC2"                 "snpPC3"                 "snpPC4"                
[49] "snpPC5"                 "snpPC6"                 "snpPC7"                
[52] "snpPC8"                 "snpPC9"                 "snpPC10"               
[55] "ageGroup"               "ageStage"               "Fetal_replicating"     
[58] "Fetal_quiescent"        "OPC"                    "Neurons"               
[61] "Astrocytes"             "Oligodendrocytes"       "Microglia"             
[64] "Endothelial"            "fetal"                  "birth"                 
[67] "infant"                 "child"                  "teen"                  
[70] "adult"                  "mean_mitoRate"          "mean_totalAssignedGene"
[73] "mean_RIN"

## Clean data and save

### Using received data

In [ ]:
load("../data/rse_gene_BrainSeq_phases_1_2.n1627.Rdata")
colData(rse_gene) %>% data.frame %>% group_by(Dataset,Dx) %>% count()

In [124]:
rse_gene %>% colData %>% colnames

[1] "SAMPLE_ID"         "RNum"              "RIN"              
 [4] "Region"            "Dataset"           "BrNum"            
 [7] "Dx"                "Age"               "Sex"              
[10] "Race"              "Protocol"          "numReads"         
[13] "numMapped"         "numUnmapped"       "mitoMapped"       
[16] "totalMapped"       "overallMapRate"    "concordMapRate"   
[19] "mitoRate"          "rRNA_rate"         "totalAssignedGene"
[22] "bamFile"

In [139]:
rse_gene %>% colData %>% data.frame %>% group_by(Dataset) %>% count()

Dataset,n
<chr>,<int>
BrainSeq_Phase1,727
BrainSeq_Phase2_DLPFC,453
BrainSeq_Phase2_HIPPO,447


In [142]:
rse_gene_filter <- rse_gene %>% 
    subset(
        select = 
            # (Dx %in% c('SCZD')) & 
            (Dataset %in% c('BrainSeq_Phase2_DLPFC')) &
            (Age >= 13) & (Age <= 150)
    )

In [144]:
library(DESeq2)
assays(rse_gene_filter)['vst'] <- varianceStabilizingTransformation(assay(rse_gene_filter))

converting counts to integer mode

Warning message in recycleSingleBracketReplacementValue(value, x, nsbs):
“number of values supplied is not a sub-multiple of the number of values to be replaced”


In [ ]:
assays(rse_gene)['vst'] %>% 
data.frame %>% rownames_to_column('ensembl_id') %>% 
write_csv('../outputs/exp_SCZ.csv')

### Using public data

In [55]:
rse_merge_vars <- rse_gene %>% merge_covariates
rse_select_samples <- rse_merge_vars %>% select_samples(diagnosis=c('Schizo'))
rse_clean_samples <- rse_select_samples %>% clean_outlier_samples
rse_clean_genes <- rse_clean_samples %>% clean_genes
rse_cell_proportions <- rse_clean_genes %>% get_cell_proportions
rse_norm_samples <- rse_cell_proportions %>% normalise_samples

Merged data from multi-lane samples.
Selected 254 of 900 samples.
Removed 6 outliers from 248 samples.
Filtered 37223 of 58037 genes.
Added cell proportion estimates from BRETIGEA.
Normalised logRPKM quantiles using preprocessCore.


In [59]:
rse_norm_samples %>% colData() %>% data.frame %>% colnames

[1] "SAMPLE_ID"         "FQCbasicStats"     "perBaseQual"      
 [4] "perTileQual"       "perSeqQual"        "perBaseContent"   
 [7] "GCcontent"         "Ncontent"          "SeqLengthDist"    
[10] "SeqDuplication"    "OverrepSeqs"       "AdapterContent"   
[13] "KmerContent"       "percentGC_R1"      "phred100_R1"      
[16] "phredGT30_R1"      "phredGT35_R1"      "Adapter88_R1"     
[19] "percentGC_R2"      "phred100_R2"       "phredGT30_R2"     
[22] "phredGT35_R2"      "Adapter88_R2"      "ERCCsumLogErr"    
[25] "bamFile"           "trimmed"           "numReads"         
[28] "numMapped"         "numUnmapped"       "overallMapRate"   
[31] "concordMapRate"    "totalMapped"       "mitoMapped"       
[34] "mitoRate"          "totalAssignedGene" "rRNA_rate"        
[37] "RNum"              "BrNum"             "Region"           
[40] "RIN"               "Age"               "Sex"              
[43] "Race"              "Dx"                "snpPC1"           
[46] "snpPC2"            "snpPC3"            "snpPC4"           
[49] "snpPC5"            "snpPC6"            "snpPC7"           
[52] "snpPC8"            "snpPC9"            "snpPC10"          
[55] "ast"               "end"               "mic"              
[58] "neu"               "oli"               "opc"              
[61] "ast_unscaled"      "end_unscaled"      "mic_unscaled"     
[64] "neu_unscaled"      "oli_unscaled"      "opc_unscaled"

In [80]:
source("../code/preprocess.r")
rse_merge_vars <- rse_gene %>% merge_covariates
rse_select_samples <- rse_merge_vars %>% select_samples(diagnosis=c('Control', 'Schizo'))
rse_clean_samples <- rse_select_samples %>% clean_outlier_samples
rse_clean_genes <- rse_clean_samples %>% clean_genes
rse_cell_proportions <- rse_clean_genes %>% get_cell_proportions
rse_norm_samples <- rse_cell_proportions %>% normalise_samples
saveRDS(rse_norm_samples, "../data/rse_norm_samples_CTLSCZ.rds")

formula <- "~ Dx + Age + Sex + Race + mitoRate + rRNA_rate + totalAssignedGene + RIN +
                         snpPC1 + snpPC2 + snpPC3 + snpPC4 + snpPC5 + 
                         snpPC6 + snpPC7 + snpPC8 + snpPC9 + snpPC10 +
                         ast + end + mic + neu + oli + opc
                         "
rse_regressed <- rse_norm_samples %>% 
    regress_covariates(rse_tx, formula = formula, n_qsvs = NULL, P=1)
rse_norm_genes <- rse_regressed %>% normalise_genes

saveRDS(rse_norm_genes, "../outputs/rse_processed_CTL_SCZ_QSV.rds")

rse_QSV <- readRDS("../outputs/rse_processed_CTL_SCZ_QSV.rds")


Merged data from multi-lane samples.
Selected 341 of 900 samples.
Removed 7 outliers leaving 334 samples.
Filtered 36779 of 58037 genes.
Added cell proportion estimates from BRETIGEA.
Normalised logRPKM quantiles using preprocessCore.
Regressing covariates...
... matched gene & transcript samples
... made qSVs
... computed residuals
Regressed covariates.
Normalised genes by rank using RNOmni.


In [66]:
# rse_QSV_SCZ <- readRDS("../outputs/rse_processed_QSV_SCZ.rds")
assays(rse_QSV)[['ranknorm']] %>% t %>% magrittr::set_rownames(rse_QSV$BrNum) %>% 
data.frame %>% rownames_to_column('BrNum') %>% write_csv('../outputs/libd.csv')
# data.frame %>% rownames_to_column('ensembl_id') %>% t

# write_csv('../outputs/exp_SCZ.csv')

In [ ]:
test <- load("../data/methprop_pd.Rdata")

In [ ]:
pd %>% data.frame %>% tibble %>% colnames

## PCA

In [67]:
rse_QSV <- readRDS("../outputs/rse_processed_CTL_SCZ_QSV.rds")

In [82]:
exp <- assays(rse_QSV)[['ranknorm']]

In [86]:
pca <- exp %>% t %>% prcomp

In [109]:
cols <- c('RIN','Age','Sex','Race','Dx')
df <- pca$x[,1:2] %>% data.frame %>% merge(colData(rse_QSV)[,cols])

DataFrame with 111556 rows and 7 columns
             PC1        PC2       RIN       Age         Sex        Race
       <numeric>  <numeric> <numeric> <numeric> <character> <character>
1       -62.4758    9.25154       6.7     64.08           M        CAUC
2        10.8520   31.61884       6.7     64.08           M        CAUC
3       -30.7977   31.14089       6.7     64.08           M        CAUC
4        17.5558   44.12307       6.7     64.08           M        CAUC
5       -13.5833  -17.71664       6.7     64.08           M        CAUC
...          ...        ...       ...       ...         ...         ...
111552  -19.2039 44.6744819       8.9   60.5629           M        CAUC
111553   28.0671 24.5324788       8.9   60.5629           M        CAUC
111554   14.7685  0.0226641       8.9   60.5629           M        CAUC
111555  -13.3182 -7.4113561       8.9   60.5629           M        CAUC
111556  -53.0257  4.1757255       8.9   60.5629           M        CAUC
                Dx
    

In [116]:
# pca$sdev[1:2]**2/sum(pca$sdev**2)
summary(pca)

Importance of components:
                            PC1      PC2      PC3      PC4      PC5      PC6
Standard deviation     23.76733 21.60692 19.75702 18.88862 18.59010 18.33406
Proportion of Variance  0.02679  0.02214  0.01851  0.01692  0.01639  0.01594
Cumulative Proportion   0.02679  0.04893  0.06744  0.08436  0.10075  0.11669
                            PC7      PC8      PC9     PC10    PC11    PC12
Standard deviation     17.65742 17.06664 16.45297 16.24690 15.7073 15.2304
Proportion of Variance  0.01479  0.01381  0.01284  0.01252  0.0117  0.0110
Cumulative Proportion   0.13148  0.14529  0.15813  0.17065  0.1824  0.1933
                           PC13     PC14     PC15     PC16     PC17    PC18
Standard deviation     14.88926 14.87510 14.24993 13.86195 13.59793 13.3905
Proportion of Variance  0.01051  0.01049  0.00963  0.00911  0.00877  0.0085
Cumulative Proportion   0.20387  0.21436  0.22399  0.23310  0.24187  0.2504
                           PC19     PC20     PC21     PC22    

## WGCNA

In [ ]:
rse_QSV <- readRDS("../outputs/rse_processed_QSV.rds")
rse_noQSV <- readRDS("../outputs/rse_processed_noQSV.rds")

In [ ]:
source("../code/wgcna.r")
# sft <- get_soft_threshold(rse_noQSV)
# power <- sft$powerEstimate

net_QSV <- fit_WGCNA(rse_QSV, power=3, threads = 9,
                loadTOM = TRUE, fileBase = "QSV")

net_noQSV <- fit_WGCNA(rse_noQSV, power=4, threads = 9,
                loadTOM = TRUE, fileBase = "noQSV")